In [1]:
from pyscf import gto, lib
import numpy as np

np.set_printoptions(16, suppress=False, linewidth=300)

In [2]:
mol1 = gto.Mole(atom="O; H 1 0.94; H 1 0.94 2 104.5", basis="def2-TZVP").build()

In [3]:
basis = {
    "C": "cc-pVTZ",
    "H": "cc-pVTZ",
    "Sb": "cc-pVTZ-PP",
}

basis_ecp = {
    "Sb": "cc-pVTZ-PP",
}

mol2 = gto.Mole(atom="""
Sb        -1.33937843      0.44597852     -1.27279684
Sb         1.33937843     -0.44597852     -1.27279684
C         -1.40429524      1.10441871      0.83468205
C         -2.16210130     -1.56132398     -0.84717555
C          2.16210130      1.56132398     -0.84717555
C          1.40429524     -1.10441871      0.83468205
H         -0.69918639      1.91987631      1.00872018
H         -1.16111477      0.29030616      1.51873028
H         -2.40124532      1.47235562      1.08516843
H         -2.02002046     -2.22909286     -1.69887295
H         -1.69052287     -2.01612927      0.02577778
H         -3.23450854     -1.49489801     -0.65423339
H          2.02002046      2.22909286     -1.69887295
H          3.23450854      1.49489801     -0.65423339
H          1.69052287      2.01612927      0.02577778
H          0.69918639     -1.91987631      1.00872018
H          2.40124532     -1.47235562      1.08516843
H          1.16111477     -0.29030616      1.51873028
""", basis=basis, ecp=basis_ecp).build()

## concat molecules

In [4]:
mol = mol1 + mol1

In [5]:
lib.fp(mol._atm), lib.fp(mol._bas), lib.fp(mol._env), lib.fp(mol._ecpbas)

(220.2819072746202, 149.65011901575278, -7210.728997361069, 0.0)

In [6]:
mol = mol1 + mol2

In [7]:
lib.fp(mol._atm), lib.fp(mol._bas), lib.fp(mol._env), lib.fp(mol._ecpbas)

(-68.54444636475233,
 -58.06413507172557,
 26407.797117689446,
 164.89860996992238)

In [8]:
mol = mol2 + mol1

In [9]:
lib.fp(mol._atm), lib.fp(mol._bas), lib.fp(mol._env), lib.fp(mol._ecpbas)

(-714.6455874682379, -99.88753072795852, -32210.1073858801, 110.04541227168156)

In [10]:
mol = mol2 + mol2

In [11]:
lib.fp(mol._atm), lib.fp(mol._bas), lib.fp(mol._env), lib.fp(mol._ecpbas)

(1355.7032887239495, -589.9932899367334, 1519.3261601043766, 779.9379788456364)

## fakemol for charges

In [12]:
coords = [[0.0, 0.0, 0.0], [1.0, 0.0, 0.0], [2.0, 1.5, 2.3], [3.0, 4.0, 5.0]]
exponent = 0.5
fakemol = gto.fakemol_for_charges(np.array(coords), exponent)

In [13]:
lib.fp(fakemol._atm), lib.fp(fakemol._bas), lib.fp(fakemol._env), lib.fp(fakemol._ecpbas)

(80.41184821237731, 3.9145826029379323, 1.4694722804235696, 0.0)

In [14]:
out = gto.intor_cross("int1e_ovlp", mol1, fakemol)
lib.fp(out.T), out.shape

(0.3218956097258599, (43, 4))

## with-clauses

In [15]:
mol = mol1.copy()

### common_orig

In [16]:
assert abs(lib.fp(mol.intor("int1e_r")) - -0.7587292491644675) < 1e-10

In [17]:
with mol.with_common_orig([0, 1, 2]):
    assert abs(lib.fp(mol.intor("int1e_r")) - 71.88577867872883) < 1e-10

### rinv_orig

In [18]:
assert abs(lib.fp(mol.intor("int1e_rinv")) - 51.806443495904794) < 1e-10

In [19]:
with mol.with_rinv_orig([0, 1, 2]):
    assert abs(lib.fp(mol.intor("int1e_rinv")) - 15.72929399764994) < 1e-10

### rinv_at_nucleus

In [20]:
assert abs(lib.fp(mol.intor("int1e_rinv")) - 51.806443495904794) < 1e-10

In [21]:
with mol.with_rinv_at_nucleus(1):
    assert abs(lib.fp(mol.intor("int1e_rinv")) - 20.940503856155193) < 1e-10

### with_range_coulomb

In [22]:
assert abs(lib.fp(mol.intor("int2e")) - 70.00106603114841) < 1e-10

In [23]:
with mol.with_range_coulomb(0.5):
    assert abs(lib.fp(mol.intor("int2e")) - 23.8282413132626) < 1e-10

In [24]:
with mol.with_range_coulomb(-0.5):
    assert abs(lib.fp(mol.intor("int2e")) - 46.17282471793578) < 1e-10

In [25]:
with mol.with_long_range_coulomb(0.5):
    assert abs(lib.fp(mol.intor("int2e")) - 23.8282413132626) < 1e-10

In [26]:
with mol.with_short_range_coulomb(0.5):
    assert abs(lib.fp(mol.intor("int2e")) - 46.17282471793578) < 1e-10

### rinv_at_nucleus (ECP)

In [27]:
mol = mol2.copy()

In [28]:
assert abs(lib.fp(mol.intor("ECPscalar_iprinvip")) - 324.13737563392084) < 1e-10

In [29]:
with mol.with_rinv_at_nucleus(1):
    assert abs(lib.fp(mol.intor("ECPscalar_iprinvip")) - 302.6772698217352) < 1e-10